In [ ]:
path = "C:/Users/pietr/Desktop/Python/Work_Project/simplewiki-2020-11-01.jsonl.gz"

In [ ]:
json_filename = "simplewiki-2020-11-01.jsonl"

In [ ]:
# Check if file exists
if not os.path.exists(path):
    raise FileNotFoundError(f"Error: The file '{path}' does not exist.")

with gzip.open(
    path, "rt", encoding="utf-8"
) as f:  # Use gzip.open() for compressed files
    data = [json.loads(line) for line in f]  # Read each line as JSON

if len(data) > 10:
    print(data[10])  # Print the 11th JSON object
else:
    print("Warning: Not enough lines in the dataset!")

In [ ]:
df = pd.DataFrame(data)

df

In [ ]:
# Check if 'paragraphs' column exists and drop NaN values
# if "paragraphs" not in df.columns:
# raise KeyError("Error: 'paragraphs' column not found in DataFrame!")

if "paragraphs" in df and not df["paragraphs"].isna().all():
    df["paragraphs"] = df["paragraphs"].dropna().reset_index(drop=True)

In [ ]:
df = df.dropna(subset=["paragraphs"])  # Remove rows with NaN values

# Load Sentence Transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")  # Load a model (modify if needed)

# Encode paragraphs
corpus_embedding = model.encode(df["paragraphs"].tolist(), convert_to_tensor=True)

In [ ]:
# define the queries to test the model

queries = ["Kevin Dunn"]

corpus = df["paragraphs"].tolist()

top_k = min(5, len(corpus))  # It determines the smaller of two values:
# 5 and len(corpus), where len(corpus) represents the length of the list, string, or iterable named corpus.

# initiate a loop to show the 5 closest results to each query, taken from the corpus

for query in queries:
    query_embedding = model.encode(query, convert_to_tensor=True)
    manual_similarities = util.pytorch_cos_sim(query_embedding, corpus_embedding)[0]
    # model.similarity(query_embedding, corpus_embedding) likely returns a single array of similarity scores for the query.
    scores, indices = torch.topk(manual_similarities, k=top_k)
    print("\nQuery:", query)
    print("Top 5 most similar sentences in corpus:")
    for score, idx in zip(scores, indices):
        print(df.iloc[idx]["paragraphs"], f"(Score: {score:.4f})")